# Analysis

This is my analysis of WorldBank's maternal mortality data.

## Cleaning

First, I'm going to load and clean the data. Loading the data is just going to involve standard
`pathlib` operations. In place of null values, the data uses '..'. So first, I'm going to convert those into
`NaN` values, before converting all of the year data into integer types.

From there, I'm going to convert the year columns into a single year column.

In [1]:
import collections
import json
from pathlib import Path
import re

import altair as alt
import numpy as np
import pandas as pd

# reduce Jupyter size by saving altair data as external urls
# https://altair-viz.github.io/user_guide/faq.html
alt.data_transformers.enable('json')

PROJ_ROOT = Path("..")
DATA_ROOT = PROJ_ROOT / "data/"
WORLDBANK_DATA = DATA_ROOT / "worldbank-data.csv"

indicator = "Maternal mortality ratio (modeled estimate, per 100,000 live births)"
all_indicators = pd.read_csv(WORLDBANK_DATA)
filter_maternal_mortality = all_indicators[all_indicators["Series Name"] == indicator].copy()

# just make sure that each country is its own row
assert len(filter_maternal_mortality["Country Name"].unique()) == len(filter_maternal_mortality)
assert len(filter_maternal_mortality["Country Code"].unique()) == len(filter_maternal_mortality)

filter_maternal_mortality.head(2)

,Series Name,Series Code,Country Name,Country Code,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
63909,"Maternal mortality ratio (modeled estimate, pe...",SH.STA.MMRT,Afghanistan,AFG,905,858,810,786,701,673,638,..,..,..
63910,"Maternal mortality ratio (modeled estimate, pe...",SH.STA.MMRT,Albania,ALB,22,17,16,16,15,16,15,..,..,..


In [2]:
year_filter = filter_maternal_mortality.filter(regex=r"(?P<year>[0-9]{4}) \[YR(?P=year)\]")
    
maternal_mortality = filter_maternal_mortality.melt(
    id_vars=["Country Name", "Country Code"],
    value_vars=year_filter.columns,
    var_name="year",
    value_name="mortality_rate"
)
maternal_mortality["mortality_rate"] = maternal_mortality.mortality_rate.replace(
    "..", np.nan
).apply(float)
maternal_mortality.head(3)

,Country Name,Country Code,year,mortality_rate
0,Afghanistan,AFG,2011 [YR2011],905.0
1,Albania,ALB,2011 [YR2011],22.0
2,Algeria,DZA,2011 [YR2011],116.0


## Analysis

From here, I'm going to analyze the maternal mortality data, using altair to visually inspect the data. I'll start simply by graphing the mean mortality across all of the countries and will move on to looking at specific years.

In [3]:
mean_mortality = maternal_mortality.groupby(
    ["Country Name", "Country Code"]
).mortality_rate.mean().reset_index().dropna().sort_values(
    by="mortality_rate", ascending=False
)

alt.Chart(mean_mortality).mark_bar().encode(
    x="mortality_rate",
    y=alt.Y("Country Name", sort="-x")
)

alt.Chart(...)

The visualization suggests that a lot of the places where maternal mortality is the highest are in poorer countries or are in countries with higher levels of conflict.

I also want to break down the data year by year to see what sort of changes there have been:

In [4]:
maternal_mortality.groupby(by="year").mortality_rate.median()

year
2011 [YR2011]    72.0
2012 [YR2012]    71.0
2013 [YR2013]    70.0
2014 [YR2014]    69.0
2015 [YR2015]    68.0
2016 [YR2016]    67.0
2017 [YR2017]    67.0
2018 [YR2018]     NaN
2019 [YR2019]     NaN
2020 [YR2020]     NaN
Name: mortality_rate, dtype: float64

This suggests that maternal mortality has been decreasing overall. At this point, I want to see where maternal mortality rates have and haven't been decreasing:

In [5]:
countries_mean_mortality = mean_mortality[
    ["Country Code", "mortality_rate"]
].set_index("Country Code").to_dict("index")
mortality_notnull = maternal_mortality.dropna().copy()
mortality_notnull["country_mean"] = mortality_notnull["Country Code"].apply(
    lambda x: countries_mean_mortality[x]["mortality_rate"]
)
mortality_notnull["above_mean_mortality"] = mortality_notnull.mortality_rate > mortality_notnull.country_mean
mortality_notnull["pct_diff_mean"] = (
    mortality_notnull.mortality_rate - mortality_notnull.country_mean
) / mortality_notnull.country_mean
mortality_notnull.head(5)

,Country Name,Country Code,year,mortality_rate,country_mean,above_mean_mortality,pct_diff_mean
0,Afghanistan,AFG,2011 [YR2011],905.0,767.285714,True,0.179482
1,Albania,ALB,2011 [YR2011],22.0,16.714286,True,0.316239
2,Algeria,DZA,2011 [YR2011],116.0,114.285714,True,0.015000
5,Angola,AGO,2011 [YR2011],300.0,263.714286,True,0.137595
6,Antigua and Barbuda,ATG,2011 [YR2011],43.0,42.857143,True,0.003333


In [6]:
mortality_notnull.sort_values(by="pct_diff_mean").head(20)

,Country Name,Country Code,year,mortality_rate,country_mean,above_mean_mortality,pct_diff_mean
1596,Belarus,BLR,2017 [YR2017],2.0,3.285714,False,-0.391304
1758,Norway,NOR,2017 [YR2017],2.0,3.000000,False,-0.333333
1435,Kazakhstan,KAZ,2016 [YR2016],10.0,13.571429,False,-0.263158
1698,Kazakhstan,KAZ,2017 [YR2017],10.0,13.571429,False,-0.263158
1836,West Bank and Gaza,PSE,2017 [YR2017],27.0,34.714286,False,-0.222222
1633,Czech Republic,CZE,2017 [YR2017],3.0,3.857143,False,-0.222222
1748,Nepal,NPL,2017 [YR2017],186.0,236.000000,False,-0.211864
1711,Latvia,LVA,2017 [YR2017],19.0,24.000000,False,-0.208333
1598,Belize,BLZ,2017 [YR2017],36.0,44.857143,False,-0.197452
1749,Netherlands,NLD,2017 [YR2017],5.0,6.142857,False,-0.186047


In [7]:
mortality_notnull.sort_values(by="pct_diff_mean").tail(20)

,Country Name,Country Code,year,mortality_rate,country_mean,above_mean_mortality,pct_diff_mean
69,Estonia,EST,2011 [YR2011],13.0,10.857143,True,0.197368
187,Panama,PAN,2011 [YR2011],76.0,63.000000,True,0.206349
128,Lao PDR,LAO,2011 [YR2011],272.0,225.285714,True,0.207356
170,Nepal,NPL,2011 [YR2011],285.0,236.000000,True,0.207627
34,Cambodia,KHM,2011 [YR2011],232.0,192.000000,True,0.208333
258,West Bank and Gaza,PSE,2011 [YR2011],42.0,34.714286,True,0.209877
216,South Africa,ZAF,2011 [YR2011],161.0,133.000000,True,0.210526
281,Belarus,BLR,2012 [YR2012],4.0,3.285714,True,0.217391
248,United Arab Emirates,ARE,2011 [YR2011],4.0,3.285714,True,0.217391
511,United Arab Emirates,ARE,2012 [YR2012],4.0,3.285714,True,0.217391


In [24]:
alt.Chart(mortality_notnull).mark_point().encode(
    x="year",
    y="mortality_rate",
    tooltip=["Country Name", "country_mean"]
)

alt.Chart(...)

#### 2017

In which countries was the maternal mortality rate below the country's mean mortality rate in 2017?

In [9]:
mortality_notnull[
    (mortality_notnull.year == "2017 [YR2017]") & 
    (~mortality_notnull.above_mean_mortality)
].sort_values(by="pct_diff_mean").head(20)

,Country Name,Country Code,year,mortality_rate,country_mean,above_mean_mortality,pct_diff_mean
1596,Belarus,BLR,2017 [YR2017],2.0,3.285714,False,-0.391304
1758,Norway,NOR,2017 [YR2017],2.0,3.000000,False,-0.333333
1698,Kazakhstan,KAZ,2017 [YR2017],10.0,13.571429,False,-0.263158
1836,West Bank and Gaza,PSE,2017 [YR2017],27.0,34.714286,False,-0.222222
1633,Czech Republic,CZE,2017 [YR2017],3.0,3.857143,False,-0.222222
1748,Nepal,NPL,2017 [YR2017],186.0,236.000000,False,-0.211864
1711,Latvia,LVA,2017 [YR2017],19.0,24.000000,False,-0.208333
1598,Belize,BLZ,2017 [YR2017],36.0,44.857143,False,-0.197452
1749,Netherlands,NLD,2017 [YR2017],5.0,6.142857,False,-0.186047
1594,Bangladesh,BGD,2017 [YR2017],173.0,212.285714,False,-0.185061


#### 2016

In [10]:
mortality_notnull[
    (mortality_notnull.year == "2016 [YR2016]") & 
    (~mortality_notnull.above_mean_mortality)
].sort_values(by="pct_diff_mean").head(20)

,Country Name,Country Code,year,mortality_rate,country_mean,above_mean_mortality,pct_diff_mean
1435,Kazakhstan,KAZ,2016 [YR2016],10.0,13.571429,False,-0.263158
1507,Poland,POL,2016 [YR2016],2.0,2.428571,False,-0.176471
1459,Luxembourg,LUX,2016 [YR2016],5.0,6.000000,False,-0.166667
1573,West Bank and Gaza,PSE,2016 [YR2016],29.0,34.714286,False,-0.164609
1430,Israel,ISR,2016 [YR2016],3.0,3.571429,False,-0.160000
1485,Nepal,NPL,2016 [YR2016],200.0,236.000000,False,-0.152542
1358,Chile,CHL,2016 [YR2016],13.0,15.142857,False,-0.141509
1366,Croatia,HRV,2016 [YR2016],7.0,8.142857,False,-0.140351
1335,Belize,BLZ,2016 [YR2016],39.0,44.857143,False,-0.130573
1443,Lao PDR,LAO,2016 [YR2016],196.0,225.285714,False,-0.129994


#### 2015

In [11]:
mortality_notnull[
    (mortality_notnull.year == "2015 [YR2015]") & 
    (~mortality_notnull.above_mean_mortality)
].sort_values(by="pct_diff_mean").head(20)

,Country Name,Country Code,year,mortality_rate,country_mean,above_mean_mortality,pct_diff_mean
1244,Poland,POL,2015 [YR2015],2.0,2.428571,False,-0.176471
1196,Luxembourg,LUX,2015 [YR2015],5.0,6.000000,False,-0.166667
1167,Israel,ISR,2015 [YR2015],3.0,3.571429,False,-0.160000
1131,Finland,FIN,2015 [YR2015],3.0,3.428571,False,-0.125000
1283,Sweden,SWE,2015 [YR2015],4.0,4.571429,False,-0.125000
1172,Kazakhstan,KAZ,2015 [YR2015],12.0,13.571429,False,-0.115789
1053,Albania,ALB,2015 [YR2015],15.0,16.714286,False,-0.102564
1071,Belgium,BEL,2015 [YR2015],5.0,5.571429,False,-0.102564
1300,United Arab Emirates,ARE,2015 [YR2015],3.0,3.285714,False,-0.086957
1070,Belarus,BLR,2015 [YR2015],3.0,3.285714,False,-0.086957


#### Compared to 2011

Finally, I want to see which countries have had the largest change since 2011.

In [12]:
all_years = mortality_notnull.pivot(index="Country Name", columns="year", values="mortality_rate")
((all_years["2017 [YR2017]"] - all_years["2011 [YR2011]"]) / all_years["2011 [YR2011]"]).sort_values().head(30)

Country Name
Belarus                   -0.600000
Norway                    -0.500000
Kazakhstan                -0.473684
Luxembourg                -0.375000
West Bank and Gaza        -0.357143
Nepal                     -0.347368
Poland                    -0.333333
Lao PDR                   -0.319853
Albania                   -0.318182
Panama                    -0.315789
Cambodia                  -0.310345
Estonia                   -0.307692
Belize                    -0.307692
Bangladesh                -0.302419
Turkmenistan              -0.300000
Afghanistan               -0.295028
Rwanda                    -0.289398
Netherlands               -0.285714
Ireland                   -0.285714
Timor-Leste               -0.282828
Ethiopia                  -0.281362
Chile                     -0.277778
Latvia                    -0.269231
South Asia (IDA & IBRD)   -0.265766
South Asia                -0.265766
India                     -0.263959
South Africa              -0.260870
Russian Federat

## Overall Findings

The overall findings show that maternal mortality rates tend to be the highest in sub-Saharan African countries: in all of the years for which data is available, the countries with the three-highest rates were all in sub-Saharan Africa.

In [13]:
for year in mortality_notnull["year"].unique():
    year_data = mortality_notnull[mortality_notnull.year == year]
    highest_mortality = list(year_data.sort_values(
        by="mortality_rate", ascending=False
    ).head(6)["Country Name"])
    joined_mortality = "\n\t".join(highest_mortality)
    print(f"The highest mortality rates in {year} were:{joined_mortality}")

The highest mortality rates in 2011 [YR2011] were:Sierra Leone
	Chad
	South Sudan
	Central African Republic
	Nigeria
	Somalia
The highest mortality rates in 2012 [YR2012] were:Sierra Leone
	Chad
	South Sudan
	Central African Republic
	Nigeria
	Somalia
The highest mortality rates in 2013 [YR2013] were:Sierra Leone
	Chad
	South Sudan
	Central African Republic
	Nigeria
	Somalia
The highest mortality rates in 2014 [YR2014] were:Sierra Leone
	Chad
	South Sudan
	Central African Republic
	Nigeria
	Somalia
The highest mortality rates in 2015 [YR2015] were:Sierra Leone
	Chad
	South Sudan
	Nigeria
	Central African Republic
	Somalia
The highest mortality rates in 2016 [YR2016] were:Chad
	South Sudan
	Sierra Leone
	Nigeria
	Central African Republic
	Somalia
The highest mortality rates in 2017 [YR2017] were:South Sudan
	Chad
	Sierra Leone
	Nigeria
	Central African Republic
	Somalia


Overall, mortality rates appear to have dropped slightly between 2011 and 2017. And there are some countries where the drop was particularly large: In Afghanistan, for instance, mortality dropped from a rate of 905 deaths per 100,000 in 2011 to a 638 in 2017. Moreover, few countries had a higher mortality rate in 2017 than in 2011:

In [14]:
(all_years["2017 [YR2017]"] > all_years["2011 [YR2011]"]).mean()

0.06926406926406926

## Export for Visualization

Finally, I'm going to export my data for use in a visualization. This will mostly consist of reformatting the columns of the data so they can work well as JSON values.

In [22]:
def export_data(df: pd.DataFrame, output_file: Path):
    """Exports a DataFrame into a file for visualization."""
    viz_export = df.rename(
        columns={k: int(re.search(r"(^[0-9]{4})", k).group(1)) for k in df.columns}
    )
    export_data = viz_export.to_dict("split")
    export_data["countries"]= export_data["index"]
    export_data["years"] = export_data["columns"]
    del export_data["index"]
    del export_data["columns"]
    export_data.update({
        "min_mortality": viz_export.min().min(),
        "max_mortality": viz_export.max().max(),
    })
    with open(output_file, "w") as f:
        json.dump(export_data, f)
        
income = all_years.loc[
    [
    "Low income",
     "Low & middle income", 
     "Lower middle income",
     "Middle income",
     "Upper middle income",
     "High income"
    ]
]

decreasing_mortality = all_years.loc[
    [
        "Afghanistan",
        "Nepal",
        "Lao PDR",
        "Cambodia",
        "Bangladesh",
        "Rwanda",
        "Timor-Leste",
        "Ethiopia",
        "India",
        "South Africa",
        "Mozambique"
    ]
]

export_data(income, "../gender-data/public/income.json")
export_data(decreasing_mortality, "../gender-data/public/decreasing-mortality.json")